In [1]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
!wget https://storage.googleapis.com/control-meme-public/meme_variation_20230311-120313.274010.jpeg -O example.jpeg


--2024-07-06 19:01:11--  https://storage.googleapis.com/control-meme-public/meme_variation_20230311-120313.274010.jpeg
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.204.91, 172.217.169.27, 172.217.169.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.204.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23289 (23K) [image/jpeg]
Saving to: ‘example.jpeg’

example.jpeg        100%[===================>]  22.74K  --.-KB/s    in 0.09s   

2024-07-06 19:01:11 (259 KB/s) - ‘example.jpeg’ saved [23289/23289]



In [24]:
import requests
import os
from jinja2 import Environment, FileSystemLoader
import dotenv
import json
dotenv.load_dotenv()

# complete workflow templates
def fill_template(template_name: str, **kwargs) -> dict:
    file_loader = FileSystemLoader("../workflows")
    env = Environment(loader=file_loader)

    # Load the template
    template = env.get_template(f"{template_name}.json.j2")

    # Render the template with the data
    rendered_json = template.render(kwargs)
    print(rendered_json)
    print(type(rendered_json) )
    # convert str to dict
    return json.loads(rendered_json)


# call comfyUI runpod api
def comfy_workflow(b64_input: str, workflow: dict) -> str:
    runpod_api_key = os.environ["RUNPOD_API_KEY"]
    runpod_endpoint_id = os.environ["RUNPOD_ENDPOINT_ID"]

    res = requests.post(
        url=f"https://api.runpod.ai/v2/{runpod_endpoint_id}/runsync",
        headers={"Authorization": f"Bearer {runpod_api_key}"},
        json={
            "input": {
                "workflow": workflow,
                "images": [{"name": "example.png", "image": b64_input}],
            }
        },
    )

    return res.json()

In [4]:
from PIL import Image
import base64
# load exemple image in b64
with open("example.jpeg", "rb") as image_file:
    b64_image = base64.b64encode(image_file.read()).decode("utf-8")

In [27]:
seed = 123
prompt = "A beautiful sunset over the ocean."

generation_workflow = fill_template("generation", seed=seed, prompt=prompt)
comfy_reply = comfy_workflow(b64_image, generation_workflow)


{
    "3": {
      "inputs": {
        "seed": 123,
        "steps": 20,
        "cfg": 8,
        "sampler_name": "euler",
        "scheduler": "normal",
        "denoise": 1,
        "model": [
          "4",
          0
        ],
        "positive": [
          "13",
          0
        ],
        "negative": [
          "7",
          0
        ],
        "latent_image": [
          "5",
          0
        ]
      },
      "class_type": "KSampler",
      "_meta": {
        "title": "KSampler"
      }
    },
    "4": {
      "inputs": {
        "ckpt_name": "sd_xl_base_1.0.safetensors"
      },
      "class_type": "CheckpointLoaderSimple",
      "_meta": {
        "title": "Load Checkpoint"
      }
    },
    "5": {
      "inputs": {
        "width": 512,
        "height": 512,
        "batch_size": 1
      },
      "class_type": "EmptyLatentImage",
      "_meta": {
        "title": "Empty Latent Image"
      }
    },
    "6": {
      "inputs": {
        "text": "A beautiful sunse

In [34]:
from io import BytesIO

In [36]:
b64_output = comfy_reply["output"]["message"]
image_data = base64.b64decode(b64_output)
pil_image = Image.open(BytesIO(image_data))


In [37]:
comfy_reply.keys()

dict_keys(['delayTime', 'executionTime', 'id', 'output', 'status'])